# The Knapsack Problem

| value | description |
| :---- | :-------------------- |
| $N$ | number of objects |
| $a$ | indice of objects |
| $W$ | weight limit the knapsack can carry |
| $c_{a}$ | value of each object |
| $w_{a}$ | weight of each object |
| $x_{a}$ | binary variable <br> 1 &rarr; object is in the knapsack <br> 0 &rarr; object is not in the knapsack |
| $y_{n}$ | binary variable <br> 1 &rarr; the final weight is $n$ <br> 0 &rarr; the final weight it not $n$ |

We have a knapsack/bag that can hold a maximal weight $W$. We also have $N$ objects, lables by the indice $a$, each having an own weight $w_{a}$ and a value $c_{a}$. The binary variable $x_{a}$ shows whether the object is in the knapsack/bag being 1 or not being 0.
We wish to maximize $c$ and have the constraint $w \le W$.
We will write our cost function as $H = H_{A} + H_{B}$, where

$H_{A}$ ensures that the weight can only take on one value and it equals the value we claim it does.
$H_{A} = A (1 - \sum_{n=1}^W y_{n})^2 + A (\sum_{n=1}^W ny_{n} - \sum_{a}w_{a}x_{a})^2$

$H_{B}$ aims at maximizing the value being held by the knapsack/bag. <br>
$H_{B} = -B \sum_{a}c_{a}x_{a}$

Additionally adding one object to the knapsack/bag which will lead to $n > W$ should not be allowed. Therefore we require $0 < B max(c_{a}) < A$.

To connect to the Azure Quantum workspace you've previously deployed you will need to import the needed libraries.
Get the workspace information from the Overview if your workspace in the portal. You can also retreive this infotmation by running 'az quantum workspace show'. Copy them here:

In [ ]:
from azure.quantum.optimization import Term
from math import floor, log2
from azure.quantum import Workspace

workspace = Workspace (
  subscription_id = "",
  resource_group = "",
  name = "",
  location = ""
)

Derived by Lucas paper we are going to implement $W = (W +1 - 2^M) + \sum_{i=0}^{M-1} 2^i$. <br>
$W = (W +1 - 2^M)$ represents the last step. <br>

**costsArray** contains $c_{a}$ (values of each object) <br>
**costsArray** contains $w_{a}$ (weights of each object) <br>
**maxCosts** is the maximal $w_{a}$ in the costsArray <br>
**n** equals $N$ (number of objects) <br>
**M** is the sqare root of $W$ (weight limit the knapsack can carry) <br>
**i** is an indice<br>
**j** is an indice<br>
**k** is the formular to encode $W$ by the auxiliary variable y<br>
**y** is an auxiliary variable. Defined as $y_{0} \le y_{i} \le y_{M}$<br>


In [ ]:
def knapsackHamiltonian(costsArray, weightsArray, W):
    
    terms = []

    maxCosts = max(costsArray)

    n = len(costsArray)

    M = floor(log2(W))

    k = [2**i for i in range(M)]
    
    # the mentioned last step
    k.append(W + 1 - 2**M)
    
    # x-Term
    for i in range(n):
        terms.append(Term(c=float(maxCosts * (weightsArray[i]**2) - costsArray[i]), indices=[i]))

    # x-x Term
    for i in range(n):
        for j in range(i+1, n):
            terms.append(Term(c=float(2*maxCosts*weightsArray[i]*weightsArray[j]), indices=[i,j]))

    # x-y Term
    for i in range(n):
        for j in range(M+1):
            terms.append(Term(c=float(-2*maxCosts*weightsArray[i]*k[j]), indices=[i,(n-1)+j]))

    # y Term
    for i in range(M+1):
        terms.append(Term(c=float(maxCosts*(k[i]**2)), indices=[(n-1)+i]))
        

    # y-y Term
    for i in range(M+1):
        for j in range(i+1, M+1):
            terms.append(Term(c=float(2*maxCosts*k[i]*k[j]), indices=[(n-1)+i,(n-1)+j]))

    return terms

Import the modules needed to run your problem. Change the imported Solver if you wish to solve with a different method.


In [ ]:
from azure.quantum.optimization import Problem, ProblemType
from azure.quantum.optimization import SimulatedAnnealing

Here you can add the values $c$ and weights $w$ of your objects and the maximum weight $W$ the knapsack/bag can hold.

In [ ]:
costsArray = [5, 4, 2, 1, 4]
weightsArray = [7, 2, 5, 6, 8]
Weight = 15

Here you initialize the term formulation and define the parameters for the problem and the solver.

In [ ]:
terms = knapsackHamiltonian(costsArray, weightsArray, Weight)
problem = Problem(name="knapsack problem", problem_type=ProblemType.pubo, terms=terms)
solver = SimulatedAnnealing(workspace, timeout=100, seed=22)

Here you submit the terms to the solver in form of a job. You request the results from the job ans display them.

In [ ]:
job = solver.submit(problem)
job.refresh()
result = job.get_results()
config = result['configuration']
resultitems = []
for i in config.keys():
    if config[i]:
         resultitems.append(int(i))
    
for i in resultitems:
    if i < len(costsArray):
        print("Picked item number " + str(i))